In [1]:
dataset_dir="/workspace/data/thermix_data/thermset_v1.1"
group="Fiona_001"
day=2
start_hour=12

modelname="2016-11-04_145241"
checkpoint=6097
#modelname="2016-11-11_073507"
#checkpoint=2622
modelpath="/workspace/data/thermix_data/tf_training/%s/checkpoint-%d"%(modelname,checkpoint)
tf_workspace ="/workspace/data/thermix_data/tf_evaluation"

fps=30

In [2]:
import os

input_filepath=os.path.join(tf_workspace,"%s_%s_day%d.in"%(group, modelname, day))
output_filepath=os.path.join(tf_workspace,"%s_%s_day%d.out"%(group, modelname, day))

video_filepath=os.path.join(tf_workspace,"%s_%s_day%d.mov"%(group, modelname, day))

In [3]:
from datetime import datetime, timedelta

annotations_filename = "%s_annotations.txt"%group
annotations_filepath = os.path.join(dataset_dir, group, annotations_filename)

clips_path = os.path.join(dataset_dir,group)
clips = [(c, datetime.strptime(c,"%Y-%m-%d_%H.%M.%S")) for c in os.listdir(clips_path) if c != annotations_filename]

clips = sorted(clips, key=lambda c:c[1])

start_date=(clips[0][1]+timedelta(days=day-1)).replace(hour=start_hour, minute=0, second=0)
end_date=start_date+timedelta(days=1)
print start_date, end_date 

clips_to_process=[]
for c,d in clips:
    if start_date <= d <= end_date:
        clips_to_process.append(c)
      
input_dir = os.path.dirname(input_filepath)
if not os.path.isdir(input_dir):
    os.makedirs(input_dir)
  
frames_to_process = []
for c in clips_to_process:
    c_path = os.path.join(clips_path, c)
    frames=sorted(os.listdir(c_path), key=lambda x:int(x[:-4]))
    for fr in frames:
        fr_path=os.path.join(c_path, fr)
        frames_to_process.append(fr_path)
            
with open(input_filepath, "w") as f:
    for fr_path in frames_to_process:
        f.write("%s 0\n"%fr_path)

print "frames to predict: %d"%len(frames_to_process)
print frames_to_process[:100]

2016-10-21 15:00:00 2016-10-22 15:00:00
frames to predict: 25926
['/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/0.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/1.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/2.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/3.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/4.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/5.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/6.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/7.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.00.43/8.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.01.33/0.png', '/workspace/data/thermix_data/thermset_v1.1/Marge_012/2016-10-21_15.01.33/1.png', '/workspace/data/thermix_data/th

In [33]:
import tflearn, tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import batch_normalization#local_response_normalization
#from tflearn.layers.estimator import regression
from training_utils import thermal_image_preloader

def model(input_placeholder=None):
    tf_data = input_placeholder or tf.placeholder(tf.float32, shape=(None, 224, 224))
    network = input_data(placeholder=tf_data)
    
    network = reshape(network, [-1,224,224,1])
    
    network = batch_normalization(network)
    
    network = conv_2d(network, 96, 7, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, strides=2, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 384, 3, activation='relu')
    network = batch_normalization(network)
    
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = batch_normalization(network)
    
    #network = local_response_normalization(network)
    network = fully_connected(network, 5, activation='softmax')
    
    return network, tf_data

graph = tf.Graph()
with graph.as_default():
    # Model variables
    #The network to be used
    with tf.device('/gpu:0'):
        net, X_ph = model()

        init = tf.initialize_all_variables()
        vars_to_restore = []
        for v in tf.all_variables():
            if "is_training" in v.name:
                print v.name
                continue
            vars_to_restore.append(v)
        saver = tf.train.Saver(vars_to_restore)

X, _ =thermal_image_preloader(input_filepath, (224,224), mode='file', normalize=False,
                              grayscale=True, categorical_labels=True,
                              files_extension=None, filter_channel=False)

# Launch the graph
with tf.Session(graph=graph, config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
    tflearn.config.is_training(False,sess)
    sess.run(init)
    saver.restore(sess,modelpath)
    
    prediction = []
    i=0
    step = 200
    while len(prediction) <len(X):
        res = net.eval({X_ph:X[i:i+step]})
        for r in res:
            sorted_indexes = sorted(range(5), key=lambda x:r[x], reverse=True)
            prediction.append((sorted_indexes[0], r[sorted_indexes[0]]))
        i+=step

BatchNormalization/is_training:0


In [34]:
#save results
with open(output_filepath, "w") as f:
    for i in range(len(frames_to_process)):
        fr_path = frames_to_process[i]
        label = prediction[i][0]
        score = prediction[i][1]
        f.write("%s %d:%f\n"%(fr_path,label,score))

In [35]:
from draw_utils import draw_classification_on_image
import shutil

temp_dir= os.path.join(tf_workspace, "temp")
if os.path.isdir(temp_dir):
    shutil.rmtree(temp_dir)

os.makedirs(temp_dir)

labels=["Sleeping","Sitting","Standing","People","Background"]
for i in range(len(frames_to_process)):
    
    fr = frames_to_process[i]
    score = prediction[i][1]
    label = labels[prediction[i][0]]
    draw_classification_on_image(fr, os.path.join(temp_dir, "%020d.png"%i), score, label, stretch_image=True, add_date=True)

In [6]:
if os.path.exists(video_filepath):
    os.remove(video_filepath)
    
os.system("ffmpeg -pattern_type glob -i '%s/*.png' -r %d -c:v libx264 %s" % (temp_dir, fps, video_filepath))
#shutil.rmtree(temp_dir)
os.system("cd youtube_upload && python upload_video.py --file=%s --title='%s Day %d' && cd .."% (video_filepath, group, day))

0

In [36]:
from datetime import datetime, timedelta

def str_date(date):
    return date.strftime("%Y-%m-%d %H:%M:%S")

with open(output_filepath) as f:
    lines=f.readlines()

video_lens = {}
for line in lines:
    fr,info = line.split()
    
    video_name= os.path.basename(os.path.dirname(fr))
    if not video_lens.get(video_name):
        video_lens[video_name]= 1
    else:
        video_lens[video_name]+= 1
    
category_lines = {}
cat_by_index = ["lying", "sitting", "standing", "people","background"]
last_cat=None
for line in lines:
    fr,info = line.split()
    cat,score=info.split(":")
    str_cat=cat_by_index[int(cat)]
    
    video_name= os.path.basename(os.path.dirname(fr))
    frame_index=int(os.path.basename(fr)[:-4])
    
    start_date= str_date(datetime.strptime(video_name,"%Y-%m-%d_%H.%M.%S")+timedelta(seconds=(float(frame_index)/video_lens[video_name])*5))
    
    if not category_lines.get(str_cat):
        category_lines[str_cat]=[]
    
    if not last_cat:
        category_lines[str_cat].append(start_date)
        last_cat = str_cat
        continue
        
    if last_cat != str_cat:
        category_lines[last_cat].extend([start_date,None])
        category_lines[str_cat].append(start_date)
        last_cat = str_cat
        
    if line == lines[-1]:
        category_lines[str_cat].append(start_date)

for c in category_lines.keys():
    if category_lines[c][-1] is None:
        category_lines[c].pop(-1)

In [42]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
cat_order = ["background","lying", "sitting", "standing", "people"]
for cat in cat_order:
    trace_p = go.Scatter(
        x = category_lines[cat],
        y = [cat for _ in category_lines[cat]],
        mode = 'lines',
        name = '%s' %(cat)
    )
    
    data.append(trace_p)

layout = dict(title='%s Day %d timeline'%(group, day))
fig = dict(data=data, layout=layout)
    
py.iplot(fig, filename='%s Day %d timeline'%(group, day))

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

